Notebook for second problem

In [1]:
import pandas as pd

Reading in Data and storing codon to aminoacid mapping in a dictionary.

In [2]:
codonData=pd.read_table("RNA_codon_table_1.txt",sep=" ",header=None)
codonData.head()

,0,1
0,AAA,K
1,AAC,N
2,AAG,K
3,AAU,N
4,ACA,T


Getting codon mappings to protein

In [3]:
def getCodonMappings(row,mappings={}):
    '''Gets the DNA codon to corresponding Amino Acid mappings and stores it in a Dictionary'''
    val=row[0].replace('U','T')
    mappings[val]=row[1]
#Hashtable to store mappings
geneticCode={}
#Applying function to dataframe to get mappings
codonData.apply(getCodonMappings,mappings=geneticCode,axis=1);#Semicolon suppresses output

Defining method for getting reverseComplement of DNA

In [4]:
reverseBases={'A':'T','T':'A','G':'C','C':'G'}
def getReverseComplement(inputDNA):
    '''Gets the reverse complement of a given DNA string'''
    reverseDNA=""
    for base in inputDNA[::-1]:
        reverseDNA+=reverseBases[base]
    return(reverseDNA)        

Testing reverseComplement function

In [5]:
getReverseComplement("ATGC")

'GCAT'

Reading input file

In [6]:
inputFile=open("dataset_96_8.txt")
inputLines=inputFile.read().splitlines()
inputDNA=inputLines[0].strip()
candidatePeptide=inputLines[1].strip()

Printing an excerpt to check

In [7]:
print(inputDNA[1:10])
print(candidatePeptide)

GCAGGCTAC
FFCCSCDF


Defining all the high level methods required to get what I want

In [8]:
def getTranslatedProtein(inputText,geneticCode):
    '''Converts a given DNA sequence into its corresponding peptide sequence using the genetic code
    '''
    codonLocationTuples=zip(range(0,len(inputText)-3+1,3),range(3,len(inputText)+1,3))
    outputString=""
    for locs in codonLocationTuples:
        codon=(inputText[locs[0]:locs[1]])
        translatedCodon=geneticCode[codon]
        #If we encounter a stop codon we stop translation of genetic code
        if type(translatedCodon) is float:
            break
        else:
            outputString+=geneticCode[codon]
    return(outputString)

def getReturnSequences(inputDNA,peptide):
    '''Returns the return DNA sequences for all 3 forward reading frames on inputDNA 
    '''
    returnSequences=[]
    lengthofDNA=len(peptide)*3
    startIndices=range(0,len(inputDNA)-lengthofDNA+1)
    endIndices=range(lengthofDNA,len(inputDNA)+1)
    startandEndPairs=zip(startIndices,endIndices)
    for pairs in startandEndPairs:
        dna=inputDNA[pairs[0]:pairs[1]]
        translatedPeptide=getTranslatedProtein(dna,geneticCode)
        if translatedPeptide==peptide:
            returnSequences.append(dna)
    return(returnSequences)

Opening output file for writing

In [9]:
outputFile=open("output.txt",'w')

Writting results for forward matching sequences found for forward strand

In [10]:
forwardSequences=getReturnSequences(inputDNA,candidatePeptide)
for seq in forwardSequences:
    outputFile.write(seq+"\n")

Writting results for reverse matching sequences found in reverse strand

In [11]:
reverseSequences=getReturnSequences(getReverseComplement(inputDNA),candidatePeptide)
for seq in reverseSequences:
    outputFile.write(getReverseComplement(seq)+"\n")

Closing input and output files

In [12]:
outputFile.close()
inputFile.close()